## Final Project - Reveal AI

In [1]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn

In [2]:
# !pip install timm

In [3]:
size_check = {}
# verify that the images are the same size or not
# print(os.listdir('1_fake')[0:10])
for img in os.listdir('data/1_fake'):
    if '.ipynb_checkpoints' == img:
        continue
    images=Image.open('data/1_fake/'+img)
    # print(images.size)
    if images.size in size_check:
        size_check[images.size] += 1
    else:
        size_check[images.size] = 1
print(list(size_check.items())[0:10])
        

[((450, 675), 1301), ((764, 1019), 419), ((764, 974), 1), ((600, 800), 8), ((512, 768), 275), ((764, 1242), 43), ((346, 520), 1), ((764, 1221), 5), ((764, 573), 227), ((1024, 1536), 295)]


In [4]:
size_check = {}
# verify that the images are the same size or not
# print(os.listdir('1_fake')[0:10])
for img in os.listdir('data/0_real'):
    if '.ipynb_checkpoints' == img:
        continue
    images=Image.open('data/0_real/'+img)
    # print(images.size)
    if images.size in size_check:
        size_check[images.size] += 1
    else:
        size_check[images.size] = 1
print(list(size_check.items())[0:10])
        

[((500, 333), 4795), ((500, 750), 3014), ((500, 301), 8), ((500, 732), 5), ((500, 334), 572), ((500, 625), 579), ((500, 415), 1), ((500, 749), 326), ((500, 332), 237), ((500, 327), 11)]


In [5]:
# Resize images for convnext
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(root='data', transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

print(f"Classes: {dataset.classes}") 
print(f"Total images: {len(dataset)}")

Classes: ['0_real', '1_fake']
Total images: 26035


In [6]:

model = timm.create_model('convnext_tiny', pretrained=True, num_classes=2)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

print("Model ready on device:", device)

Model ready on device: cpu


In [7]:


optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

for epoch in range(2): 
    total, correct, running_loss = 0, 0, 0
    model.train()

    for imgs, labels in dataloader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = 100 * correct / total
    print(f"Epoch {epoch+1}: Loss={running_loss/len(dataloader):.4f}, Acc={acc:.2f}%")

KeyboardInterrupt: 

In [ ]:

# random image from fake set
img_path = 'data/1_fake/0ac485de-d5f7-4ee5-a620-354b6b766314.jpg'

# same transform as training
tf = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

img = Image.open(img_path).convert("RGB")
x = tf(img).unsqueeze(0).to(device)

model.eval()
with torch.no_grad():
    pred = model(x).argmax(1).item()

label_names = dataset.classes  
print("Predicted class:", label_names[pred])

plt.imshow(img)
plt.title(f"Predicted: {label_names[pred]}")
plt.axis("off")
plt.show()